# Retrievers

In [ ]:
# change working directory to the root directory, since the notebook is inside a folder
import os
os.chdir(os.path.join(os.getcwd(),os.pardir))

In [ ]:
# read the .env file for environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# enable langsmith tracing
# all the runs will be traced in the default project since we have not explicitely defined a project name
# langsmith - https://smith.langchain.com/
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [1]:
# create a web base loader from the langsmith documentation
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [2]:
type(docs)

list

In [3]:
docs[:1]

[Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly un

In [4]:
# we will need to divide this data into smaller chunks and store their embeddings in a vectorstore
# for this, we will need a model to create embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
# we will now split the documents, create embeddings and store in the vectorstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [6]:
# now we will create a prompt, and merge the prompt and llm to create a document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# initiate llm
# we can use llms from multiple sources
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
# check the datatype of llm object
type(llm)

langchain_openai.chat_models.base.ChatOpenAI

In [ ]:
# we can directly use this llm object by send a query
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools, such as unit testing frameworks, integration testing tools, and end-to-end testing libraries. These tools can help developers quickly and efficiently test their code, identify bugs and issues, and ensure that their software functions as intended. Additionally, Langsmith can provide testing best practices, guidelines, and documentation to help developers effectively test their code and improve the quality of their software.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 15, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30834c07-01e4-4695-8cf8-41bb7f9be372-0')

In [ ]:
# we can improve the response by creating a prompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

# creating an llm chain
chain = prompt | llm 

In [ ]:
# checking the data type of the chain
type(chain)

langchain_core.runnables.base.RunnableSequence

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to generate test cases and test data automatically, which can help speed up the testing process and ensure comprehensive test coverage.\n\n2. Test Scenario Generation: Langsmith can create various test scenarios based on the specifications provided, helping testers to identify edge cases and potential vulnerabilities.\n\n3. Regression Testing: Langsmith can assist in creating regression test suites to ensure that new changes do not introduce any unintended side effects or bugs in the existing codebase.\n\n4. Load Testing: Langsmith can generate realistic user scenarios to simulate high loads on the system, helping to identify performance bottlenecks and scalability issues.\n\n5. API Testing: Langsmith can be used to generate API requests and responses for testing API endpoints, ensuring that the API functions correctly under various conditions.\n\nOverall, Langsmith can en

In [ ]:
# the output is not in a human readable format. we can use output parsers to convert this into a string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# creating a new chain with output parse
chain = prompt | llm | output_parser

In [ ]:
result = chain.invoke({"input": "how can langsmith help with testing?"})
print(result)

Langsmith is a powerful tool that can greatly assist with testing by providing various features and capabilities that streamline the testing process. Here are some ways in which Langsmith can help with testing:

1. **Automated Testing:** Langsmith can be used to automate the testing of code by writing test cases in a structured format. This can help ensure that code changes do not introduce new bugs or regressions.

2. **Test Data Generation:** Langsmith can generate test data that can be used to validate the functionality of the software under test. This can help in ensuring comprehensive test coverage.

3. **Code Coverage Analysis:** Langsmith can analyze code coverage during testing, helping to identify areas of code that have not been adequately tested. This can assist in improving the effectiveness of testing efforts.

4. **Integration Testing:** Langsmith can be integrated with other testing tools and frameworks to perform integration testing of different components of the softwa